<a href="https://colab.research.google.com/github/shm0007/sexist-comment-detection/blob/main/HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Shamim Ehsan
#HW5
##04-17-2023
Fine-tuning RoBERTa-large model to detect sexist comment.
Dataset:
google colab link: https://colab.research.google.com/drive/1egtPVZYXXWJDBQmvAsiJU0JmPEF-RqYq?usp=sharing

In [ ]:
!pip install datasets transformers[sentencepiece]
!pip install evaluate
import transformers

In [ ]:
import csv,json

# opening the CSV file
file = open('sexism_data.csv', mode ='r')
csv_reader = csv.reader(file)
next(csv_reader)

total_lines =  13632 #run another time to get the actual line count

#local variable to keep track of the rows, total sexist and not sexist comment
it = 0
sexist = 0
non_sexist = 0


#list to store train,test,validation data
train = []
test = []
validation  = []


for lines in csv_reader:

  #set the label to 0 or 1 based on 5th row of csv
  if lines[4]== "False":
    label = 0
    non_sexist +=1
  else:
    label  = 1
    sexist+=1

  #json format
  data = {"comment": lines[2], "label": label}
  it+=1
  #first 75% goes to train
  if it < total_lines * .75:
    train.append(data)
  #75%-90% data goes to validation
  elif it < total_lines*.90:
    validation.append(data)
  #remaining 10% for test
  else:
    test.append(data)

print(len(train))
print(len(validation))
print(len(test))




#write train,validation and test data into file
json_train = {"data": train}
f = open("train.json", "w",encoding="utf-8")
f.write(json.dumps(json_train))
f.close()
json_test = {"data": test}
f = open("test.json", "w",encoding="utf-8")
f.write(json.dumps(json_test))
f.close()
json_validation = {"data": validation}
f = open("validation.json", "w",encoding="utf-8")
f.write(json.dumps(json_validation))
f.close()

10223
2045
1363


In [ ]:
print(sexist)
print(non_sexist)
print((non_sexist)  / (sexist+ non_sexist))

1809
11822
0.867287799867948


In [ ]:
from datasets import load_dataset, load_metric
data_files = {"train": "train.json", "test": "test.json","validation":"validation.json"}
dataset = load_dataset("json",  data_files=data_files, field="data")

In [ ]:
from transformers import AutoTokenizer
model_checkpoint = "roberta-large"
batch_size =32
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
def tokenize_function(examples):
    return tokenizer(examples["comment"], truncation=True)
tokenized_dataset = dataset.map(tokenize_function, batched=True)
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)

In [ ]:
args = TrainingArguments(
    "roberta-sexist",
    overwrite_output_dir = True,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,0.135195
2,0.179900,0.183976
3,0.179900,0.251800
4,0.085800,0.271586
5,0.037100,0.324961


TrainOutput(global_step=1600, training_loss=0.09590421065688133, metrics={'train_runtime': 1648.3459, 'train_samples_per_second': 31.01, 'train_steps_per_second': 0.971, 'total_flos': 4419311354002020.0, 'train_loss': 0.09590421065688133, 'epoch': 5.0})

In [ ]:
trainer.save_model()

In [ ]:
import numpy as np

#predict for test dataset
poredictions = trainer.predict(tokenized_dataset['test'])
preds = np.argmax(poredictions.predictions, axis=-1)
import evaluate

#load accuracy metric
metric = evaluate.load("accuracy")
metric.compute(predictions=preds, references=poredictions.label_ids)

{'accuracy': 0.9449743213499633}

In [ ]:
from transformers import pipeline
sexist = pipeline("text-classification", model = './roberta-sexist')

In [ ]:
sexist('who wants to go to mars?')

[{'label': 'LABEL_0', 'score': 0.9997950196266174}]

In [ ]:
sexist("Real men don't cry")

[{'label': 'LABEL_1', 'score': 0.9992843270301819}]

In [ ]:
sexist("Don't be a pussy, stop crying")

[{'label': 'LABEL_1', 'score': 0.998241662979126}]

In [ ]:
sexist("Stop crying")

[{'label': 'LABEL_0', 'score': 0.9996858835220337}]

In [ ]:
sexist("You have such a skinny figure.")

[{'label': 'LABEL_0', 'score': 0.9981503486633301}]

In [ ]:
sexist("She’s such a drama queen.")

[{'label': 'LABEL_0', 'score': 0.9996336698532104}]

In [ ]:
sexist("Once married, a woman must learn to adjust")

[{'label': 'LABEL_1', 'score': 0.9996172189712524}]

In [ ]:
#Code block directly taken from tutorial to check the dataset

import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(dataset["train"])

,text,label
0,Josh & Mum ..mmmmm Yummo!!! #MKR,0
1,"MENTION4039 a lot of people contribute just a little bit, and it turns into enough to keep me afloat. every dollar is immensely appreciated.",0
2,Bugger #mkr,0
3,MENTION4683 MENTION2978 I think the hashtag got some traction. #cuntandandre #mkr,0
4,Kat & Andre cartoon characters.Kat develop sportsmanship & Andre grow some balls & take it like a man! #MKR #cantcook http://t.co/9nd2gUHGH0,1
5,I have yet to meet see or know a kid who knows how to drive or what on Earth she's doing on the road! save our roads!,0
6,"We'll paү for a girl's meal once if we're trүing to get luckү. If we do it more than a few times, we actuallү like үou.",1
7,I hate this... #nomancard RT MENTION1600: My teammates boyfriend wants to take her last name. Her last name. Equality,0
8,"""you're abnormally tall for a girl""",1
9,"""People still come chill & sex this!"".. KAPLOW!!",0
